Create an electronic report with a maximum of 2000 words (excluding citations) using Jupyter. The report should include the posed question, conducted analysis, and derived conclusion. Only one team member needs to submit this report. It is not required to include all tasks completed by each group member in their individual assignments. Make sure to reach a consensus among all team members on the final content of the report. If needed, consult your TA and Instructor for further guidance.

## Title

### Introduction

Start with relevant background information on the topic to prepare those unfamiliar for the rest of your proposal.

Formulate one or two questions for investigation and detail the dataset that will be utilized to address these questions.

Additionally, align your question/objectives with the existing literature. To contextualize your study, include a minimum of two scientific publications (these should be listed in the References section).

GitHub repositories are online platforms that are useful for coding projects and assignments. Repositories allow for collaboration with other GitHub users, as well as viewing opportunities for anyone interested in exploring topics of interest. Projects can vary based on content, coding language and size, just to name a few aspects. Furthermore, repository features allow for identifying popularity among repositories, something which will be of importance in our analysis.

Our question will be the following:

*Which explanatory variables best predict stars on a GitHub repository?*

Surveys confirm that stars are viewed by pratictioners as the most useful measure of a repository's popularity (Borges & Valente, 2018). 

### Methods and Results

In this section, you will include:

a) “Exploratory Data Analysis (EDA)”

Demonstrate that the dataset can be read into R.
Clean and wrangle your data into a tidy format.
Plot the relevant raw data, tailoring your plot to address your question.
Make sure to explore the association of the explanatory variables with the response.
Any summary tables that are relevant to your analysis.
Be sure not to print output that takes up a lot of screen space.
Your EDA must be comprehensive with high quality plots.

Describe in written English the methods you used to perform your analysis from beginning to end, and narrate the code that does the analysis.
If included, describe the “Feature Selection” process and how and why you choose the covariates of your final model.
Make sure to interpret/explain the results you obtain. It’s not enough to just say, “I fitted a linear model with these covariates, and my R-square is 0.87”.
If inference is the aim of your project, a detailed interpretation of your fitted model is required, as well as a discussion of relevant quantities (e.g., are the coefficients significant? How does the model fit the data)?
A careful model assessment must be conducted.
If prediction is the project's aim, describe the test data used or how it was created.
Ensure your tables and/or figures are labelled with a figure/table number.

* Title: Most Popular GitHub Repositories (Projects)
* Number of Observations: 215029
* Number of Variables: 24

The dataset consists of public repository data from GitHub, collected using the GitHub API. The data includes a wide range of attributes related to the repositories, such as metadata about the repository, owner details and various statistical measures about contributions and activity. Each repository corresponds to a project created by each respective user.

Here's a detailed summary of the variables in the dataset:

* `Name` - chr, name of the repository.
* `URL` - chr, URL or web address that links to the GitHub repository.
* `Created_At` - dttm, date and time when the repository was initially created on GitHub.
* `Updated_At` - dttm, date and time of the most recent update/modification to the repository.
* `Homepage` - chr, URL to the homepage or landing page associated with the repository.
* `Size` - dbl, size of the repository in bytes.
* `Stars` - dbl, number of stars or likes that the repository has received from other users.
* `Forks` - dbl, number of times the repository has been forked by other users.
* `Issues` - dbl, number of total issues.
* `Watchers` - dbl, number of GitHub users who are "watching" or monitoring the repository.
* `Language` - chr, primary coding language.
* `License` - chr, Information about the software license using a license identifier.
* `Topics` - list, a list of topics or tags associated with the repository.
* `Has_Issues` - lgl, indicates whether the repository has an issue tracker enabled.
* `Has_Projects` - lgl, indicates whether the repository uses GitHub Projects.
* `Has_Downloads` - lgl, indicates whether the repository offers downloadable files or assets.
* `Has_Wiki` - lgl, indicates whether the repository has an associated wiki.
* `Has_Pages` - lgl, indicates whether the repository has GitHub Pages enabled for a website.
* `Has_Discussions` - lgl, indicates whether the repository has GitHub Discussions enabled.
* `Is_Fork`- lgl, indicates whether the repository is a fork of another repository.
* `Is_Archived` - lgl, indicates whether the repository is archived.
* `Is_Template` - lgl, indicates whether the repository is set up as a template.
* `Default_Branch` - chr, name of the default branch.

The dataset will provide valuable insights into the associations between the outlined explanatory variables and the star count of GitHub repositories. By analyzing this data, we aim to develop models that effectively highlight which variables are most strongly associated with repository popularity. While we will not assume causality, our goal is to assess the relative importance of these explanatory variables and infer which ones demonstrate the strongest relationships with the response variable.

Our inquiry primarily focuses on inference, as we seek to draw conclusions about the broader population of GitHub repositories. Specifically, we want to determine whether our model can accurately forecast the number of stars a repository might receive based on its characteristics, even for observations outside of the dataset. The accuracy of our predictions will be crucial in evaluating the effectiveness of our analysis and confirming whether we have successfully identified the variables with the highest explanatory power. 

### Discussion

In this section, you’ll interpret the results you obtained in the previous section with respect to the main question/goal of your project.

Summarize what you found and the implications/impact of your findings.
If relevant, discuss whether your results were what you expected to find.
Discuss how your model could be improved;
Discuss future questions/research this study could lead to.

### References

At least two citations of literature relevant to the project. The citation format is your choice – just be consistent. Make sure to cite the source of your data as well.

https://www.sciencedirect.com/science/article/abs/pii/S0164121218301961